## 简述 \__init__、\__new__、\__call__ 方法
https://foofish.net/magic-method.html

任何事物都有一个从创建，被使用，再到消亡的过程，在程序语言面向对象编程模型中，对象也有相似的命运：创建、初始化、使用、垃圾回收，不同的阶段由不同的方法（角色）负责执行。

定义一个类时，大家用得最多的就是 \__init__ 方法，而 \__new__ 和 \__call__ 使用得比较少，这篇文章试图帮助大家把这3个方法的正确使用方式和应用场景分别解释一下。

关于 Python 新式类和老式类在这篇文章不做过多讨论，因为老式类是 Python2 中的概念，现在基本没人再会去用老式类，新式类必须显示地继承 object，而 Python3 中，只有新式类，默认继承了 object，无需显示指定，本文代码都是基于 Python3 来讨论。

### \__init__方法

\__init__方法负责对象的初始化，系统执行该方法前，其实该对象已经存在了，要不然初始化什么东西呢？先看例子：

In [8]:
# class A(object): python2 必须显示地继承object

class A:
    def __init__(self):
        print("__init__ ")
        super(A, self).__init__()

    def __new__(cls):
        print("__new__ ")
        return super(A, cls).__new__(cls)

    def __call__(self):  # 可以定义任意参数
        print('__call__ ')

A()

__new__ 
__init__ 


从输出结果来看， \__new__方法先被调用，返回一个实例对象，接着 \__init__ 被调用。

\__call__方法并没有被调用，这个我们放到最后说，先来说说前面两个方法，稍微改写成：

In [6]:
class A:
    def __init__(self):
        print("__init__ ")
        print(self)
        super(A, self).__init__()
        
    def __new__(cls):
        print("__new__ ")
        self =  super(A, cls).__new__(cls)
        print(self)
        return self
    
    def __call__(self):  # 可以定义任意参数
        print('__call__ ')

A()

__new__ 
__init__ 


输出结果来看，\__new__ 方法的返回值就是类的实例对象，这个实例对象会传递给 \__init__ 方法中定义的 self 参数，以便实例对象可以被正确地初始化。

如果 \__new__ 方法不返回值（或者说返回 None）那么 \__init__ 将不会得到调用，这个也说得通，因为实例对象都没创建出来，调用 init 也没什么意义，此外，Python 还规定，\__init__ 只能返回 None 值，否则报错，这个留给大家去试。

\__init__方法可以用来做一些初始化工作，比如给实例对象的状态进行初始化：

In [ ]:
def __init__(self, a, b):
    self.a = a
    self.b = b
    super(A, self).__init__()

另外，__init__方法中除了self之外定义的参数，都将与__new__方法中除cls参数之外的参数是必须保持一致或者等效。

In [10]:
class B:
    def __init__(self, *args, **kwargs):
        print("init", args, kwargs)

    def __new__(cls, *args, **kwargs):
        print("new", args, kwargs)
        return super().__new__(cls)

B(1, 2, 3)

new (1, 2, 3) {}
init (1, 2, 3) {}


### \__new__方法

一般我们不会去重写该方法，除非你确切知道怎么做，什么时候你会去关心它呢，它作为构造函数用于创建对象，是一个工厂函数，专用于生产实例对象。著名的设计模式之一，单例模式，就可以通过此方法来实现。在自己写框架级的代码时，可能你会用到它，我们也可以从开源代码中找到它的应用场景，例如微型 Web 框架 Bootle 就用到了。

In [ ]:
class BaseController(object):
    _singleton = None
    def __new__(cls, *a, **k):
        if not cls._singleton:
            cls._singleton = object.__new__(cls, *a, **k)
        return cls._singleton

这段代码出自 https://github.com/bottlepy/bottle/blob/release-0.6/bottle.py

这就是通过 __new__ 方法是实现单例模式的的一种方式，如果实例对象存在了就直接返回该实例即可，如果还没有，那么就先创建一个实例，再返回。

### \__call__ 方法

关于 \__call__ 方法，不得不先提到一个概念，就是可调用对象（callable），我们平时自定义的函数、内置函数和类都属于可调用对象，**但凡是可以把一对括号()应用到某个对象身上都可称之为可调用对象**，判断对象是否为可调用对象可以用函数 callable

如果在类中实现了 \__call__ 方法，那么**实例对象也将成为一个可调用对象**，我们回到最开始的那个例子：

In [15]:
print(callable(a))

True


a是实例对象，同时还是可调用对象，那么我就可以像函数一样调用它。试试：

In [16]:
a()

__call__ 


# 定制类
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014319098638265527beb24f7840aa97de564ccc7f20f6000


## \__getatrr__ (self,attr)

\_getattr__是python里的一个内建函数，可以很方便地动态返回一个属性；
当调用不存在的属性时，Python会试图调用__getattr__(self,attr)来获取属性，并且返回；

正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。比如定义Student类：调用name属性，没问题，但是，调用不存在的score属性，就有问题了：错误信息很清楚地告诉我们，没有找到score这个attribute。

In [1]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'

s = Student()
print(s.name)
print(s.score)

Michael


AttributeError: 'Student' object has no attribute 'score'

要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个\__getattr__()方法，动态返回一个属性。

当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性，这样，我们就有机会返回score的值：

In [2]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'
    def __getattr__(self,attr):
        if attr == 'score':
            return 99
    
s = Student()
print(s.name)
print(s.score)

Michael
99


返回函数也可以的，只是调用方法要改变。

In [9]:
class Student(object):
    def __getattr__(self,attr):
        if attr == 'age':
            return lambda:25
    
s = Student()
print(s.age())
print(s.abc)

25
None


**注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。**

此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。

要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误：

In [11]:
class Student(object):
    def __getattr__(self,attr):
        if attr == 'age':
            return 28
        raise AttributeError("'Student' object has no attribute '{}'".format(attr))
    
s = Student()
print(s.age)
print(s.abc)

28


AttributeError: 'Student' object has no attribute 'abc'

这实际上可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

这种完全动态调用的特性有什么实际作用呢？作用就是，可以针对完全动态的情况作调用。

举个例子：

现在很多网站都搞REST API，比如新浪微博、豆瓣啥的，调用API的URL类似：

http://api.server/user/friends

http://api.server/user/timeline/list

如果要写SDK，给每个URL对应的API都写一个方法，那得累死，而且，API一旦改动，SDK也要改。

利用完全动态的__getattr__，我们可以写出一个链式调用：

In [17]:
class Chain(object):
    def __init__(self,path=''):
        self._path = path
    def __getattr__(self,attr):
        return Chain('{}/{}'.format(self._path,attr))
    def __str__(self):
        return self._path
    __repr__=__str__
    
print(Chain().status)  
print(Chain().status.user.timeline.list)

/status
/status/user/timeline/list


实例 Chain()获取 status 属性，由于没有找该属性，调用\__getattr__， 其中 self._path 是空，attr 是 status 因而返回 Chain('/status'), 参数'/status'被传递给 path，赋值给self._path.


Chain().status.user.timeline.list “敲回车键”“()”和“.”和“回车键”都是从左向右的顺序运算符，类比成数学中的加和减。
步骤：

1、Chain是类名，对它进行“()”运算，即调用__init__(self)(这个有讲过哦),会生成一个实例c1，c1=Chain(path='')。

2、对实例c1进行“.”运算，增加一个“status”属性，即调用__getattr__(self, status),返回
一个新实例c2,c2=Chain(path='/status')。

3、对实例c2进行“.”运算，增加一个“user”属性，即调用__getattr__(self, user),返回
一个新实例c3,c3=Chain(path='/status/user')。

4、对实例c3进行“.”运算，增加一个“timeline”属性，即调用__getattr__(self, timeline),返回
一个新实例c4,c4=Chain(path='/status/user/timeline')。

5、对实例c4进行“.”运算，增加一个“list”属性，即调用__getattr__(self, list),返回
一个新实例c5,c5=Chain(path='/status/user/timeline/list')。

6、对实例c5进行“回车键”运算，即调用__repr__(self)，返回c5._path,即输出'/status/user/timeline/list'。


这样，无论API怎么变，SDK都可以根据URL实现完全动态的调用，而且，不随API的增加而改变！


还有些REST API会把参数放到URL中，比如GitHub的API：

`GET /users/:user/repos`

调用时，需要把:user替换为实际用户名。如果我们能写出这样的链式调用：

`Chain().users('michael').repos`

In [33]:
class Chain(object):
    def __init__(self,path=''):
        self._path = path
    def __getattr__(self,attr):
        return Chain('{}/{}'.format(self._path,attr))
    def __call__(self, param):
        return Chain('{}/:{}'.format(self._path,param))
    
    def __str__(self):
        return self._path
    
    __repr__=__str__
    
print(Chain().status)  
print(Chain().status.user.timeline.list)
print(Chain().users('michael').repos)

/status
/status/user/timeline/list
/users/:michael/repos


## \__call__

一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。

任何类，只需要定义一个\__call__()方法，就可以直接对实例进行调用。请看示例：

In [20]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __call__(self,data):
        print('[{}] My name is {}'.format(data,self.name))
    
s = Student('Jamie') # 创建实例 有了__init__方法，在创建实例的时候，就不能传入空的参数了，必须传入与__init__方法匹配的参数，但self不需要传
s('2019.03.27') # 实例调用，必须传入与__call__方法匹配的参数，但self不需要传

print('-----callable()------')
print(callable(s))
print(callable(max))
print(callable(1))
print(callable('str'))

[2019.03.27] My name is Jamie
-----callable()------
True
True
False
False


\__call__()还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有\__call__()的类实例